In [1]:
# from models import xresnet1d,fastai_model
import torch
import torch.nn as nn
from configs.fastai_configs import *
import pandas as pd
import wfdb
import skimage
from torch.utils.data import DataLoader
# from experiments.scp_experiment import SCP_Experiment
import numpy as np
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, BatchNormalization
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from tensorflow.keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import wfdb
import wfdb.processing
import ast
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import scipy
from tqdm import tqdm
import neurokit2 as nk

In [2]:
df = pd.read_csv("/Users/nisssaarg/Desktop/ecg_ptbxl_benchmarking-master/data/ptbxl/ptbxl.csv")
df = df[(df['age']<=65)&(df['sex']==1)]

In [3]:
path = '/Users/nisssaarg/Desktop/ecg_ptbxl_benchmarking-master/data/ptbxl/'
sampling_rate=500

In [4]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_hr)]
    data = np.array([signal for signal, meta in data])
    return data
X  = load_raw_data(df,sampling_rate,path)

100%|██████████| 5532/5532 [00:09<00:00, 573.06it/s]


In [5]:
Y = np.array(df[['CD','HYP','MI','NORM','STTC']])


In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [7]:
from models.fastai_model import fastai_model

experiment = 'exp1.1.1'
modelname = 'fastai_inception1d_no_residual'
pretrainedfolder = '../output/'+experiment+'/models/'+modelname+'/'
mpath='../output/' # <=== path where the finetuned model will be stored
n_classes_pretrained =5 # <=== because we load the model from exp0, this should be fixed because this depends the experiment
num_classes = 5
sampling_frequency = 500
input_shape = (5000,12)
model = fastai_model(
    modelname, 
    num_classes, 
    sampling_frequency, 
    mpath, 
    input_shape=input_shape, 
    # pretrainedfolder=pretrainedfolder,
    # n_classes_pretrained=n_classes_pretrained, 
    # pretrained=True,
    # epochs_finetuning=2,
    epochs=2
)

In [8]:
model.fit(X_train,Y_train,X_test,Y_test)

Training from scratch...
model: fastai_inception1d_no_residual


epoch,train_loss,valid_loss,time
0,1.046468,#na#,06:52
1,0.903198,#na#,06:44


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


epoch,train_loss,valid_loss,time
0,0.662033,1.731227,14:09
1,0.444907,0.273307,14:56


In [9]:
torch.save(model,'inception_male_young.pth')